In [3]:
import requests
import pandas as pd
import pprint

In [4]:
def get_stock_history(date, stock_no, retry = 5):   #從www.twse.com.tw讀取資料
    quotes = []
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, stock_no)
    r = requests.get(url)
    data = r.json()
    print(data)
    return data
    #return transform(data['data'])  #進行資料格式轉換

In [5]:
a = get_stock_history('20180101', '2330')
pprint.pprint(a)

{'stat': 'OK', 'date': '20180101', 'title': '107年01月 2330 台積電           各日成交資訊', 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'], 'data': [['107/01/02', '18,055,269', '4,188,555,408', '231.50', '232.50', '231.00', '232.50', '+3.00', '9,954'], ['107/01/03', '31,706,091', '7,504,382,512', '236.00', '238.00', '235.50', '237.00', '+4.50', '13,633'], ['107/01/04', '29,179,613', '6,963,192,636', '240.00', '240.00', '236.50', '239.50', '+2.50', '10,953'], ['107/01/05', '23,721,255', '5,681,934,695', '240.00', '240.00', '238.00', '240.00', '+0.50', '8,659'], ['107/01/08', '21,846,692', '5,281,823,362', '242.00', '242.50', '240.50', '242.00', '+2.00', '10,251'], ['107/01/09', '19,043,123', '4,588,314,012', '242.00', '242.00', '239.50', '242.00', ' 0.00', '7,124'], ['107/01/10', '25,716,220', '6,118,683,273', '241.50', '242.00', '236.00', '236.50', '-5.50', '10,534'], ['107/01/11', '32,070,338', '7,500,674,455', '235.00', '236.00', '232.50', '235.00', '-1.50', '9,199

In [32]:
x= pandas.DataFrame({ e[0]:e[1:] for e in a['data']})
#print(x)
x.index =a['fields'][1:]
#print (x);
x= x.transpose()
print(x)

                 成交股數            成交金額     開盤價     最高價     最低價     收盤價   漲跌價差  \
107/01/02  18,055,269   4,188,555,408  231.50  232.50  231.00  232.50  +3.00   
107/01/03  31,706,091   7,504,382,512  236.00  238.00  235.50  237.00  +4.50   
107/01/04  29,179,613   6,963,192,636  240.00  240.00  236.50  239.50  +2.50   
107/01/05  23,721,255   5,681,934,695  240.00  240.00  238.00  240.00  +0.50   
107/01/08  21,846,692   5,281,823,362  242.00  242.50  240.50  242.00  +2.00   
107/01/09  19,043,123   4,588,314,012  242.00  242.00  239.50  242.00   0.00   
107/01/10  25,716,220   6,118,683,273  241.50  242.00  236.00  236.50  -5.50   
107/01/11  32,070,338   7,500,674,455  235.00  236.00  232.50  235.00  -1.50   
107/01/12  23,141,291   5,463,302,207  234.50  238.00  233.50  237.00  +2.00   
107/01/15  28,576,533   6,832,851,887  240.00  240.00  238.00  240.00  +3.00   
107/01/16  23,407,632   5,609,009,540  240.00  240.50  238.00  240.50  +0.50   
107/01/17  38,118,119   9,207,582,787  2

In [60]:
def get_stock_price_month(date, stock_id):
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, str(stock_id))
    r=requests.get(url)
    data = r.json()
    df = pd.DataFrame({e[0]: e[1:]  for e in data['data']})
    df.index=data['fields'][1:]
    #df =df.transpose
    def taiwan_year_to_year(tw_year):
        dates = tw_year.split('/')
        dates[0] = str(int(dates[0])+1911)
        return '/'.join(dates)
    
    df.columns = df.columns.map(taiwan_year_to_year)
    df.columns = pd.to_datetime(df.columns,format='%Y/%m/%d')
    df = df.transpose()
    print(df)
    return df
    
    


In [20]:

import datetime 
from datetime import timedelta
import calendar

time_format_api = "%Y/%m/%d"
time_format_string = "%Y%m%d"

def taiwan_year_to_year(tw_year):
        dates = tw_year.split('/')
        dates[0] = str(int(dates[0])+1911)
        return '/'.join(dates)
    
def string_to_datetime(string_date,request=False):
    datetime_format = time_format_api if request else time_format_string
    return datetime.datetime.strptime(string_date,datetime_format )

def datetime_to_string(date):
    return date.strftime(time_format_string)

def get_stock_price_monthly(stock_id,date,end=None):
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, str(stock_id))
    r=requests.get(url)
    data = r.json()
    def process_datetime(all_data):
        all_data[0] = string_to_datetime(taiwan_year_to_year(all_data[0]),True)
        return all_data
    
    data['data']= list(map(process_datetime,data['data']))

    if end:
        dt_end = string_to_datetime(end)
        data['data']=list(filter(lambda in_data:  in_data[0]<= dt_end,data['data']))
    return data
        

def add_months(sourcedate,months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day,calendar.monthrange(year,month)[1])
    return datetime.datetime(year,month,day,0,0)
    
def get_stock_price(stock_id, from_date, to_date=None):
    date_from =string_to_datetime(from_date)
    if to_date:
        date_to = string_to_datetime(to_date)
    else:
        date_to = datetime.datetime.now()
    date_check = date_from
    prices= {'data':[]}
    while date_check<= date_to:
        price_data = get_stock_price_monthly(stock_id, datetime_to_string(date_check),datetime_to_string(date_to))
        prices['data'].extend(price_data['data'])
        if 'fields' not in prices:
            prices['fields'] = price_data['fields']
        date_check = add_months(date_check,1)
    return prices
        
        
#get_stock_price_monthly(2330,'20180101','20180105')
data = get_stock_price(2330,'20180101','20180212')
pprint.pprint(data)



{'data': [[datetime.datetime(2018, 1, 2, 0, 0),
           '18,055,269',
           '4,188,555,408',
           '231.50',
           '232.50',
           '231.00',
           '232.50',
           '+3.00',
           '9,954'],
          [datetime.datetime(2018, 1, 3, 0, 0),
           '31,706,091',
           '7,504,382,512',
           '236.00',
           '238.00',
           '235.50',
           '237.00',
           '+4.50',
           '13,633'],
          [datetime.datetime(2018, 1, 4, 0, 0),
           '29,179,613',
           '6,963,192,636',
           '240.00',
           '240.00',
           '236.50',
           '239.50',
           '+2.50',
           '10,953'],
          [datetime.datetime(2018, 1, 5, 0, 0),
           '23,721,255',
           '5,681,934,695',
           '240.00',
           '240.00',
           '238.00',
           '240.00',
           '+0.50',
           '8,659'],
          [datetime.datetime(2018, 1, 8, 0, 0),
           '21,846,692',
           '5,281,823

In [23]:
def get_price_report(stock_id, from_date, to_date=None):
    prices_dict = get_stock_price(stock_id, from_date,to_date)
    df = pd.DataFrame({ e[0]: e[1:] for e in prices_dict['data']}, index=prices_dict['fields'][1:])
    df = df.transpose()
    print(df)
    
get_price_report(2330,'20180101','20180312')

                   成交股數            成交金額     開盤價     最高價     最低價     收盤價  \
2018-01-02   18,055,269   4,188,555,408  231.50  232.50  231.00  232.50   
2018-01-03   31,706,091   7,504,382,512  236.00  238.00  235.50  237.00   
2018-01-04   29,179,613   6,963,192,636  240.00  240.00  236.50  239.50   
2018-01-05   23,721,255   5,681,934,695  240.00  240.00  238.00  240.00   
2018-01-08   21,846,692   5,281,823,362  242.00  242.50  240.50  242.00   
2018-01-09   19,043,123   4,588,314,012  242.00  242.00  239.50  242.00   
2018-01-10   25,716,220   6,118,683,273  241.50  242.00  236.00  236.50   
2018-01-11   32,070,338   7,500,674,455  235.00  236.00  232.50  235.00   
2018-01-12   23,141,291   5,463,302,207  234.50  238.00  233.50  237.00   
2018-01-15   28,576,533   6,832,851,887  240.00  240.00  238.00  240.00   
2018-01-16   23,407,632   5,609,009,540  240.00  240.50  238.00  240.50   
2018-01-17   38,118,119   9,207,582,787  240.50  243.00  239.00  242.00   
2018-01-18   50,119,952  

In [62]:
dt = get_stock_price_month('20180101',2330)
dt.describe()

                  成交股數            成交金額     開盤價     最高價     最低價     收盤價   漲跌價差  \
2018-01-02  18,055,269   4,188,555,408  231.50  232.50  231.00  232.50  +3.00   
2018-01-03  31,706,091   7,504,382,512  236.00  238.00  235.50  237.00  +4.50   
2018-01-04  29,179,613   6,963,192,636  240.00  240.00  236.50  239.50  +2.50   
2018-01-05  23,721,255   5,681,934,695  240.00  240.00  238.00  240.00  +0.50   
2018-01-08  21,846,692   5,281,823,362  242.00  242.50  240.50  242.00  +2.00   
2018-01-09  19,043,123   4,588,314,012  242.00  242.00  239.50  242.00   0.00   
2018-01-10  25,716,220   6,118,683,273  241.50  242.00  236.00  236.50  -5.50   
2018-01-11  32,070,338   7,500,674,455  235.00  236.00  232.50  235.00  -1.50   
2018-01-12  23,141,291   5,463,302,207  234.50  238.00  233.50  237.00  +2.00   
2018-01-15  28,576,533   6,832,851,887  240.00  240.00  238.00  240.00  +3.00   
2018-01-16  23,407,632   5,609,009,540  240.00  240.50  238.00  240.50  +0.50   
2018-01-17  38,118,119   9,2

,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
count,22,22,22,22,22,22,22,22
unique,22,22,18,16,19,16,15,22
top,"23,141,291","9,155,080,569",240.00,257.50,238.00,242.00,+2.00,"15,826"
freq,1,1,4,3,3,3,3,1


In [7]:
help(map)

Help on class map in module builtins:

class map(object)
 |  map(func, *iterables) --> map object
 |  
 |  Make an iterator that computes the function using arguments from
 |  each of the iterables.  Stops when the shortest iterable is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.

